# YOUTUBE SCRAPER PROJECT  
##  This python script extracts data from the youtube description of each video in a playlist.  
## The limitation of this script is that **this script is specific to the description format of the videos in the playlist that I used**.
#### Credits to the [야미보이 Yummyboy Channel](https://www.youtube.com/channel/UCiiV8stcewhoCNyiErr7GtA).

In [5]:
from pyshorturl import Googl, GooglError
from googleapiclient.discovery import build
import os
import csv
import pandas as pd
import urlexpander
from time import sleep

DEVELOPER_KEY = 'YOUR_DEV_KEY' #change the dev key
youtube = build('youtube', 'v3', developerKey=DEVELOPER_KEY)
service = Googl(api_key=DEVELOPER_KEY)

playlist_id = "PL5rVz0V9BCBG_mioInHjb23t8a3EHLYlA"

vid_id = []
video_ids = []
vidURL = []
pl_titles = []

## The following section extracts the english title of the playlist from the <code>youtube API</code> result and uses it to name the local folder.

In [10]:
#------------------Get english title for folder name------------------------

results1 = youtube.playlists().list(part='snippet', maxResults=50, id=playlist_id)
res1 = results1.execute()

for plres1 in res1.get('items', []):
    pl_title = plres1['snippet']['localized']['title']
    pl_title = pl_title.split("/")
    en_title = pl_title[1].strip()
    pl_titles.append(en_title)

pl_titles

#------------------Make folder of playlist------------------------
# change username and folder in path
path = 'C:\\Users\\YOUR_NAME\\Documents\\YOUR_FOLDER'+ '\\{}'.format(en_title)
#os.mkdir(path)

## The following  section extracts the ID of each video in the playlist.

In [8]:
#-------------- Extracting IDs of each vid from playlist item --------------

results = youtube.playlistItems().list(part='snippet', maxResults=50, playlistId=playlist_id)
response = results.execute()

for response in response.get('items', []):
    vidids = response['snippet']['resourceId']['videoId']
    if vidids in video_ids:
        continue
    vidURL.append("https://youtube.com/watch?v=" + vidids)
    video_ids.append(vidids)

print(video_ids)
len(video_ids)

['Rj8Qe76WJWs', 'jSgL-IB7I2U', 'xXxKhCz-7T0', 'xuy7y8fZTJY', 'Zt_ed3qDmHw', 'lgbUTliFtrE', 's8hFYWvZzdo', 'VChhdFZcWyA']


8

## The main <code>for loop</code> in the following section uses the video IDs from the previous section to make three lists.  

## The lists are:
1. Food and price
2. Google map URL
2. Location

In [9]:
#------------- Extracting description, map URL, food, and price ------------------

parts = ['localizations', 'id']
vidres = youtube.videos().list(id=video_ids, part=parts).execute()

food_price = []
food = []
loc = []
priceKRW = []
priceUSD = []

location = []
deets = []
coords = []
Lat = []
Lon = []
index = 0
inc = 0

vidURL = []
mapURL = []
mapLOC = []
listmap_url = []

class URLExpanderError(Exception):
    pass

i = 0
for res in vidres.get('items', []):
    description = res['localizations']['en']['description']
    startLOC = description.find("Location: ") + len("Location: ")
    #endLOC = description.find("\n", startLOC)
    endLOC = startLOC + 37
    map_url = description[startLOC:endLOC]

    startMAP_index = startLOC + len(map_url) + 2
    map_loc = description[startMAP_index:]
    curr_URL = "https://youtube.com/watch?v=" + res['id']
    end = 0
    listmap_url.append(map_url)

    #------------------------- Add coords to deets -------------------

    for x in range(0, 4):  # try 4 times
        try:
            expandedMapURL = urlexpander.expand(map_url)
            error = '__CONNECTIONPOOL_ERROR_'
            check = expandedMapURL.find(error)
            if check > 0:
                raise URLExpanderError
            print("trial", x, ":" , check, "CONNECTION SUCCESSFUL")
            str_error = None

        except URLExpanderError as str_error:
            print("Connection Error: Trial", x+1 , str_error)
            pass

        else:
            break

    start = expandedMapURL.find("/@")
    mid = expandedMapURL.find(",",start)
    end = expandedMapURL.find(",", mid+1)
    crds = expandedMapURL[start+2:end]
    split_coords = crds.split(",")
    #print(split_coords, '\n')
    i+=1

    lat = split_coords[0]
    lon = split_coords[1]
    Lat.append(lat)
    Lon.append(lon)

    mapURL.append(map_url)
    mapLOC.append(map_loc)
    vidURL.append(curr_URL)

    start = description.find("formation-") 
    if start == -1:
        start = description.find("formation -") + len("formation -")
    else:
        start = start + len("formation-")
    end = description.find("\n",start+2)
    fpname = []
    fpname.append(description[start + 2:end]) #first item
    lastitemloc = startLOC - len("location: ") - 2
    
    index = 0
    while end != lastitemloc: #if more than one items, these are the next items
        index += 1
        start = end + 1
        end = description.find("\n",start+1)
        fpname.append(description[start:end])
        
        # for restos with multiple food items, this code block adds space/blank in lieu of the video URL, map URL, and map location to avoid repeating in list
        if curr_URL in vidURL:
            vidURL.append("")
        if map_url in mapURL:
            mapURL.append("")
        if map_loc in mapLOC:
            mapLOC.append("")
        if end == lastitemloc:
            break
        
    food_price.append(fpname)

    inc += 1
    prev_URL = curr_URL

trial 0 : -1 CONNECTION SUCCESSFUL
trial 0 : -1 CONNECTION SUCCESSFUL
trial 0 : -1 CONNECTION SUCCESSFUL
trial 0 : -1 CONNECTION SUCCESSFUL
trial 0 : -1 CONNECTION SUCCESSFUL
trial 0 : -1 CONNECTION SUCCESSFUL
trial 0 : -1 CONNECTION SUCCESSFUL
trial 0 : -1 CONNECTION SUCCESSFUL


## From the list of food and price in the prev section, this section splits the list and make the following lists:
1. Food
2. Price in KRW
3. Price in USD.

In [59]:
# ------------------- GETTING LIST OF FOOD, KRW PRICE, AND USD PRICE ---------------------
food = []
priceKRW = []
priceUSD = []
fplen = len(food_price)

for x in range(fplen):
    f_plen = len(food_price[x])
    priceKRWlist = []
    priceUSDlist = []
    for y in range(f_plen):
        fp = food_price[x][y].split(":")
        food.append(fp[0])
        #------- split USD and KRW prices ----------
        price = fp[1].split("(")
        priceKRW.append(price[0].strip())
        priceUSD.append(price[1].rstrip(")"))
    #priceKRW.append(priceKRWlist)
    #priceUSD.append(priceUSDlist)
    deets.append([])
    deets[x].append(food[x])

print("List of food:")
print(food) 
print("List of prices in KRW:")
print(priceKRW)
print("List of prices in USD:")
print(priceUSD)

FOOD
['Margherita Pizza', 'Stellar Bulgogi Pizza', 'meat Tteokbokki', 'Assorted fried food', 'green tea ice cream strawberry crepe', 'nutella strawberry banana crepe', 'pork belly ', '1 box of sun shaped bread ', 'korean wine rice cake (50 pieces)', 'Cream Cheese Garlic Bread', 'Yangju Egg Fried Rice ']
Prices in KRW
['KRW 16,800', 'KRW 20,800', 'KRW 18,000', 'KRW 7,500', 'KRW 6,500', 'KRW 5,000', 'Samgyeopsal', 'KRW 15,000', 'KRW 14,000', 'KRW 4,800', 'KRW 9,500']
Prices in USD
['USD 15.0', 'USD 18.6', 'USD 16.0', 'USD 6.7', 'USD 5.8', 'USD 4.5', '300g', 'USD 13.4', 'USD 12.5', 'USD 4.2', 'USD 8.5']


## Finally, the section below assembles all the lists of data and create a single .csv file with all the details.

In [55]:
#------------------------------------ Writing details CSV ------------------------------------

dict = {'Food' : food,  'Price (KRW)' : priceKRW, 'Price (USD)' : priceUSD, 'Location' : mapLOC, 'Map URL' : mapURL, 'Video URL' : vidURL}
details = pd.DataFrame(dict)
details.to_csv(os.path.join(path,r'{}.csv'.format(en_title)))
